In [ ]:
import glob
import os

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from geopandas.plotting import plot_polygon_collection

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
def rename(df):
    return df.rename(columns={
        "ORIGIN_ZONE_CODE": "O_GEOGRAPHY_CODE", 
        "DESTINATION_ZONE_CODE": "D_GEOGRAPHY_CODE",
        "GENERALISED_TRAVEL_COST": "ACCESSIBILITY"
    })

In [ ]:
def no_negative(df):
    df.loc[df.ACCESSIBILITY < 0, "ACCESSIBILITY"] = df.ACCESSIBILITY.max()
    return df

In [ ]:
rail_baseline = no_negative(rename(pd.read_csv('../simim/data/arc/costRailBaseline.csv'))) \
  .rename(columns={"ACCESSIBILITY": "RAIL_BASE"})
road_baseline = no_negative(rename(pd.read_csv('../simim/data/arc/costRoadBaseline.csv'))) \
  .rename(columns={"ACCESSIBILITY": "ROAD_BASE"})
rail_scenario = no_negative(rename(pd.read_csv('../simim/data/arc/costRailScenario.csv'))) \
  .rename(columns={"ACCESSIBILITY": "RAIL_SCEN"})
road_b3 = no_negative(rename(pd.read_csv('../simim/data/arc/costRoadB3.csv'))) \
  .rename(columns={"ACCESSIBILITY": "ROAD_SCEN"})

In [ ]:
costs = rail_baseline \
  .merge(road_baseline, on=["O_GEOGRAPHY_CODE", "D_GEOGRAPHY_CODE"]) \
  .merge(rail_scenario, on=["O_GEOGRAPHY_CODE", "D_GEOGRAPHY_CODE"]) \
  .merge(road_b3, on=["O_GEOGRAPHY_CODE", "D_GEOGRAPHY_CODE"])
costs.head()

In [ ]:
costs.sort_values(by='ROAD_BASE').head(20)

In [ ]:
# smaller beta means steeper decay function (-1 falls off more slowly than -2)
beta = -3

costs["ROAD_ACCESSIBILITY_BASE"] = np.power(costs.ROAD_BASE, beta)
costs["RAIL_ACCESSIBILITY_BASE"] = np.power(costs.RAIL_BASE, beta)
costs["ACCESSIBILITY_BASE"] = np.log(np.exp(costs.ROAD_ACCESSIBILITY_BASE) + np.exp(costs.RAIL_ACCESSIBILITY_BASE))

costs["ROAD_ACCESSIBILITY_SCEN"] = np.power(costs.ROAD_SCEN, beta)
costs["RAIL_ACCESSIBILITY_SCEN"] = np.power(costs.RAIL_SCEN, beta)
costs["ACCESSIBILITY_SCEN"] = np.log(np.exp(costs.ROAD_ACCESSIBILITY_SCEN) + np.exp(costs.RAIL_ACCESSIBILITY_SCEN))

In [ ]:
# normalise to 1 maximum
max_access = max(costs.ACCESSIBILITY_BASE.max(), costs.ACCESSIBILITY_SCEN.max())
min_access = min(costs.ACCESSIBILITY_BASE.min(), costs.ACCESSIBILITY_SCEN.min())
costs["ACCESSIBILITY_BASE"] = (costs.ACCESSIBILITY_BASE - min_access) / (max_access - min_access)
costs["ACCESSIBILITY_SCEN"] = (costs.ACCESSIBILITY_SCEN - min_access) / (max_access - min_access)

In [ ]:
costs.ACCESSIBILITY_SCEN.describe()

In [ ]:
costs["RAIL_ACCESSIBILITY_DIFF"] = costs.RAIL_ACCESSIBILITY_SCEN - costs.RAIL_ACCESSIBILITY_BASE
costs["ROAD_ACCESSIBILITY_DIFF"] = costs.ROAD_ACCESSIBILITY_SCEN - costs.ROAD_ACCESSIBILITY_BASE
costs["ACCESSIBILITY_DIFF"] = costs.ACCESSIBILITY_SCEN - costs.ACCESSIBILITY_BASE

In [ ]:
costs.sort_values(by="ROAD_ACCESSIBILITY_DIFF").tail()

In [ ]:
costs.sort_values(by="RAIL_ACCESSIBILITY_DIFF").tail()

In [ ]:
costs.sort_values(by="ACCESSIBILITY_DIFF").tail()

### Force self-access to be 1

In [ ]:
costs.loc[(costs.O_GEOGRAPHY_CODE == costs.D_GEOGRAPHY_CODE), ['ACCESSIBILITY_BASE', 'ACCESSIBILITY_SCEN']] = 1

## Output to CSV

In [ ]:
baseline = costs[[
    "O_GEOGRAPHY_CODE", "D_GEOGRAPHY_CODE", 
    "ACCESSIBILITY_BASE", "ROAD_ACCESSIBILITY_BASE", "RAIL_ACCESSIBILITY_BASE"]] \
  .rename(columns={
    "ROAD_ACCESSIBILITY_BASE": "ROAD_ACCESSIBILITY", 
    "RAIL_ACCESSIBILITY_BASE": "RAIL_ACCESSIBILITY", 
    "ACCESSIBILITY_BASE": "ACCESSIBILITY"}) 

baseline.to_csv("../simim/data/access_baseline_road_rail.csv")

In [ ]:
# output with projected year in scenario (road, rail, both)

scenario = costs[[
    "O_GEOGRAPHY_CODE", "D_GEOGRAPHY_CODE", 
    "ACCESSIBILITY_SCEN", "ROAD_ACCESSIBILITY_SCEN", "RAIL_ACCESSIBILITY_SCEN"]] \
  .rename(columns={
    "ROAD_ACCESSIBILITY_SCEN": "ROAD_ACCESSIBILITY", 
    "RAIL_ACCESSIBILITY_SCEN": "RAIL_ACCESSIBILITY", 
    "ACCESSIBILITY_SCEN": "ACCESSIBILITY"})

scenario["YEAR"] = 2030

scenario.to_csv("../simim/data/scenarios/od_rail_b1.csv")

In [ ]:
baseline[baseline.O_GEOGRAPHY_CODE != baseline.D_GEOGRAPHY_CODE].sort_values(by='ACCESSIBILITY').tail()

## Effect on jobs accessibility

In [ ]:
df_emp = pd.read_csv("../simim/data/arc/arc_employment__baseline.csv")
df_gva = pd.read_csv("../simim/data/arc/arc_gva__baseline.csv")

# merge to single dataframe
df = df_gva.merge(
df_emp, on=["timestep", "lad_uk_2016"], how="left"
)

zone_base = df.reset_index().rename(columns={
"timestep": "YEAR", 
"lad_uk_2016": "GEOGRAPHY_CODE", 
"employment": "JOBS", 
"gva": "GVA", 
"gva_per_sector": "GVA"
})[[
 "YEAR", "GEOGRAPHY_CODE", "JOBS", "GVA"
]]
zone_base["GVA"] = zone_base["GVA"].round(6)
# convert from 1000s jobs to jobs
zone_base["JOBS"] = (zone_base["JOBS"] * 1000).round().astype(int)
zone_base = zone_base[zone_base.YEAR == 2030].drop("YEAR", axis=1)
zone_base.head()

In [ ]:
key = '0-unplanned'

if key == "3-new-cities23":
    econ_key = "1-new-cities"
elif key == "4-expansion23":
    econ_key = "2-expansion"
else:
    econ_key = key
df_gva = pd.read_csv("../simim/data/arc/arc_gva__{}.csv".format(econ_key))
df_emp = pd.read_csv("../simim/data/arc/arc_employment__{}.csv".format(econ_key))

# merge to single dataframe
zone_scen = df_gva \
.merge(df_emp, on=["timestep", "lad_uk_2016"], how="left") \
.rename(columns={"timestep": "YEAR", "lad_uk_2016": "GEOGRAPHY_CODE", "gva_per_sector": "GVA",
                 "employment": "JOBS",  "dwellings": "HOUSEHOLDS"})


zone_scen["GVA"] = zone_scen["GVA"].round(6)
zone_scen["JOBS"] = (zone_scen["JOBS"] * 1000).round().astype(int)  # convert from 1000s jobs to jobs
zone_scen = zone_scen[zone_scen.YEAR == 2030].drop("YEAR", axis=1)
zone_scen.head()

In [ ]:
def access_weighted_sum(dataset, colname, access_colname):
    new_colname = "D_{}_{}".format(colname, access_colname)
    # access to x[o] for each o,d 
    dataset[new_colname] = dataset["O_" + colname] * dataset[access_colname]
    # sum over o - grouping by d
    wsum = dataset[["D_GEOGRAPHY_CODE", new_colname]].groupby("D_GEOGRAPHY_CODE").sum().reset_index()

    # merge back
    dataset = dataset.merge(wsum, on="D_GEOGRAPHY_CODE") \
        .drop(new_colname + "_x", axis=1) \
        .rename({new_colname + "_y": new_colname}, axis=1)
    return dataset

In [ ]:
dataset = costs[['O_GEOGRAPHY_CODE', 'D_GEOGRAPHY_CODE', 'ACCESSIBILITY_BASE','ACCESSIBILITY_SCEN']].copy() \
  .merge(zone_base, left_on='D_GEOGRAPHY_CODE', right_on='GEOGRAPHY_CODE') \
  .rename(columns={'GVA': 'D_GVA_BASE', 'JOBS': 'D_JOBS_BASE'}) \
  .merge(zone_base, left_on='O_GEOGRAPHY_CODE', right_on='GEOGRAPHY_CODE') \
  .rename(columns={'GVA': 'O_GVA_BASE', 'JOBS': 'O_JOBS_BASE'}) \
  .merge(zone_scen, left_on='D_GEOGRAPHY_CODE', right_on='GEOGRAPHY_CODE') \
  .rename(columns={'GVA': 'D_GVA_SCEN', 'JOBS': 'D_JOBS_SCEN'}) \
  .merge(zone_scen, left_on='O_GEOGRAPHY_CODE', right_on='GEOGRAPHY_CODE') \
  .rename(columns={'GVA': 'O_GVA_SCEN', 'JOBS': 'O_JOBS_SCEN'}) \
  .drop(['GEOGRAPHY_CODE_x', 'GEOGRAPHY_CODE_y'], axis=1)

# EX LONDON
# dataset.loc[dataset.D_GEOGRAPHY_CODE.str.startswith('E09'), ['D_JOBS_BASE', 'D_JOBS_SCEN']] = 0
# dataset.loc[dataset.O_GEOGRAPHY_CODE.str.startswith('E09'), ['O_JOBS_BASE', 'O_JOBS_SCEN']] = 0

dataset = access_weighted_sum(dataset, 'JOBS_BASE', 'ACCESSIBILITY_BASE')
dataset = access_weighted_sum(dataset, 'JOBS_SCEN', 'ACCESSIBILITY_BASE')
dataset = access_weighted_sum(dataset, 'JOBS_SCEN', 'ACCESSIBILITY_SCEN')
dataset.columns

## Explore decay function

In [ ]:
lads = gpd.read_file('../simim/data/cache/Local_Authority_Districts_December_2016_Ultra_Generalised_Clipped_Boundaries_in_Great_Britain.shp')

In [ ]:
access_df = dataset[['D_GEOGRAPHY_CODE', 'D_JOBS_BASE', 'D_GVA_BASE', 'D_GVA_SCEN', 'D_JOBS_SCEN', 
  'D_JOBS_BASE_ACCESSIBILITY_BASE', 'D_JOBS_SCEN_ACCESSIBILITY_BASE', 'D_JOBS_SCEN_ACCESSIBILITY_SCEN']] \
  .copy().drop_duplicates()
access_df = lads.merge(access_df, right_on="D_GEOGRAPHY_CODE", left_on="lad16cd")
access_df['JOBS_ACCESS_DIFF'] = access_df.D_JOBS_SCEN_ACCESSIBILITY_BASE - access_df.D_JOBS_BASE_ACCESSIBILITY_BASE
access_df['JOBS_ACCESS_SCEN_DIFF'] = access_df.D_JOBS_SCEN_ACCESSIBILITY_SCEN - access_df.D_JOBS_SCEN_ACCESSIBILITY_BASE
access_df['JOBS_DIFF'] = access_df.D_JOBS_SCEN - access_df.D_JOBS_BASE
len(access_df)

In [ ]:
access_df.head()

In [ ]:
access_df.plot(column='D_JOBS_BASE_ACCESSIBILITY_BASE')

In [ ]:
def diff_plot(df, colname):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
    ax.set_aspect('equal')
    net_positive = df[df[colname] > 0.0]
    net_negative = df[df[colname] <= 0.0]

    plot_polygon_collection(
        ax, 
        net_positive['geometry'],
        clim=(0, np.max(net_positive[colname])), 
        cmap="Reds",
        values=np.abs(net_positive[colname])
    )
    plot_polygon_collection(
        ax, 
        net_negative['geometry'],
        clim=(0, np.max(np.abs(net_negative[colname]))), 
        cmap="Blues",
        values=np.abs(net_negative[colname])
    )

    return ax

In [ ]:
ax = diff_plot(access_df, 'JOBS_DIFF')
ax.set_ylim([70000, 350000])
# ax.set_ylim([270000, 650000])
ax.set_xlim([330000, 670000])
ax

In [ ]:
ax = diff_plot(access_df, 'JOBS_ACCESS_DIFF')
ax.set_ylim([70000, 350000])
# ax.set_ylim([270000, 650000])
ax.set_xlim([330000, 670000])
ax.set_title("Change in job-accessibility over baseline")
ax

In [ ]:
ax = diff_plot(access_df, 'JOBS_ACCESS_SCEN_DIFF')
ax.set_ylim([70000, 350000])
# ax.set_ylim([270000, 650000])
ax.set_xlim([330000, 670000])
ax.set_title("Difference in job-accessibility between scenarios with and without new transport links")
ax

In [ ]:
access_df[['JOBS_ACCESS_DIFF', 'JOBS_ACCESS_SCEN_DIFF', 'JOBS_DIFF']].describe()

In [ ]:
access_df[access_df.JOBS_ACCESS_SCEN_DIFF < -0.2]

In [ ]:
from_df = no_travel[no_travel.O_GEOGRAPHY_CODE == "E07000178"]
from_df = lads.merge(from_df, right_on="D_GEOGRAPHY_CODE", left_on="lad16cd")

In [ ]:
ax = from_df.plot(column="ACCESSIBILITY")
ax.set_ylim([70000, 350000])
# ax.set_ylim([270000, 650000])
ax.set_xlim([330000, 670000])
ax

In [ ]:
factor = 20
steps = factor * 9 + 1

df = pd.DataFrame({"p": list(range(factor,steps+factor))})
df.p /= factor
df["d-0.5"] = np.power(df.p, -0.5)
df["d-1.0"] = np.power(df.p, -1.0)
df["d-2.0"] = np.power(df.p, -2.0)
df["d-4.0"] = np.power(df.p, -4.0)
df["d-8.0"] = np.power(df.p, -8.0)
df.plot(x="p")